In [20]:
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
from sklearn.model_selection import train_test_split
import pandas as pd
import torch

In [21]:
df = pd.read_csv('../data/preprocessed.tsv', sep='\t')
df.head()

,Unnamed: 0.1,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,0,0,"if Alkar floods her with her mental waste, it ...","If Alkar is flooding her with psychic waste, t...",0.785171,0.010309,0.981983,0.014195
1,1,1,you're becoming disgusting.,Now you're getting nasty.,0.749687,0.071429,0.999039,0.065473
2,3,3,"monkey, you have to wake up.","Ah! Monkey, you've got to snap out of it.",0.664333,0.309524,0.994215,0.053362
3,4,4,I have orders to kill her.,I've got orders to put her down.,0.726639,0.181818,0.999348,0.009402
4,5,5,I'm not gonna have a child... ...with the same...,I'm not going to breed kids with a genetic dis...,0.703185,0.206522,0.950956,0.035846


In [22]:
df = df.sample(frac=0.8, random_state=13)

In [23]:
df = df.rename(columns={"reference": "input_text", "translation": "target_text"})
train, test = train_test_split(df, test_size=0.2)
train = pd.DataFrame(train, columns = ['input_text','target_text'])
test = pd.DataFrame(test, columns = ['input_text','target_text'])
train.head()

,input_text,target_text
308096,"Klaus is so fixated on those wolves, And elija...","Klaus is so fixated on those wolves, and Elija..."
6276,And if you spent more time letting me and less...,and if you had spent more time letting me be.....
103993,He has a fetish for violence!,he's got an aberration for violence!
209753,"Bury them now, missy?",bury them now?
390836,"when the chains are down, big boy, you'll be d...",When the chains come off... - Jesus Christ! - ...


In [24]:
test.head()

,input_text,target_text
8349,What you shove up your ass is your own business.,what you're sticking up is your thing.
342430,Fencing my boss's dog.,Blow-drying my boss's job.
227349,but the bounty hunter quickly realised how foo...,The bounty hunter realized the absurdity of hi...
308895,"everyone has signed this petition, except for ...","Everyone's signed this petition, even the ones..."
13337,"Wow, that's fucking scary.","I mean, that's creepy."


In [29]:
model_args = Seq2SeqArgs()
model_args.eval_batch_size = 64
model_args.evaluate_during_training = False
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_seq_length = 128
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
model_args.use_early_stopping = False

model_args.do_sample = True
model_args.num_beams = 1
model_args.num_return_sequences = 1
model_args.max_length = 128
model_args.top_k = 50
model_args.top_p = 0.95
model_args.output_dir = "../models"


In [30]:
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="eugenesiow/bart-paraphrase",
    args=model_args,
    use_cuda=True
)
model.train_model(train, eval_data=test)

  0%|          | 0/258928 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/32366 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/32366 [00:00<?, ?it/s]

(64732, 0.22622192151420822)